In [1]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")
 
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / 'basline' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)


# Get one cycle

In [40]:
for row in df_imgs.iloc[4:].itertuples():
    path = row.Path
    with h5py.File(path, 'r') as f:
        markers = f['imgs'].attrs['Marker']
        imgs = f['imgs'][:]
    print(markers)
    break

['DNA' 'CycD1/CDK4' 'DNA' 'Concanavalin A' 'Phalloidin' 'WGA']


In [42]:
napari.view_image(imgs, channel_axis=0, name=markers)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1737.5, 2327.0), zoom=0.242692750287687, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=2, ndisplay=2, last_used=0, range=((0.0, 3476.0, 1.0), (0.0, 4655.0, 1.0)), current_step=(1738, 2327), order=(0, 1), axis_labels=('0', '1')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'DNA' at 0x277e199d870>, <Image layer 'CycD1/CDK4' at 0x277e180a890>, <Image layer 'DNA [1]' at 0x2781b2d6920>, <Image layer 'Concanavalin A' at 0x2781b384af0>, <Image layer 'Phalloidin' at 0x278d29a2c80>, <Image layer 'WGA' at 0x278d2a4ce50>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. 

# Plot images

In [43]:
import napari
from napari_animation import Animation
from moviepy.editor import VideoFileClip

In [44]:
mask_filt_dir = data_dir /  'basline'  / 'imgs' / 'masks'

masks_path = defaultdict(dict) 
for path in os.listdir(mask_filt_dir):
    name = path.split('.')[0]
    if 'Nuclei' in name:
        masks_path[name[7:]]['nuclei'] = mask_filt_dir / path
    elif 'Cyto' in name:
        masks_path[name[5:]]['cyto'] =mask_filt_dir / path
    elif 'Cell' in name:
        masks_path[name[5:]]['cell'] =mask_filt_dir / path    
    elif 'df' in name:
        masks_path[name[3:]]['df'] =mask_filt_dir / path
    else:
        pass

In [45]:
whole_seg_path = data_dir / 'basline'/ 'imgs' / 'segmentation'

In [47]:
viewer = napari.Viewer(ndisplay=2)

for p in os.listdir(whole_seg_path):
    img = skimage.io.imread(whole_seg_path / p)
    
    name = p.split('.')[0]
    # mask_cyto_path = masks_path[name]['cell']
    mask_nuclei_path = masks_path[name]['nuclei']
    mask_cyto = skimage.io.imread(mask_cyto_path)
    mask_nuclei = skimage.io.imread(mask_nuclei_path)
    
    # Define animation layer
    viewer.add_image(img, 
                     name=name, 
                    blending='translucent',
                    rgb=True)
    # viewer.add_labels(mask_cyto, name=name+'_cyto')
    viewer.add_labels(mask_nuclei, name=name+'_nuclei')
